In [ ]:
import os
import time
import numpy as np
from glob import glob
import json
import random
import cv2
#os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
policy = tf.keras.mixed_precision.Policy("mixed_float16")
tf.keras.mixed_precision.set_global_policy(policy)
strategy = tf.distribute.MultiWorkerMirroredStrategy()
from tensorflow import keras
from tensorflow.keras import layers
from keras_cv_attention_models import efficientnet

In [ ]:
backbone_file = 'efficientnetv2-s-21k-ft1k.h5'
if not os.path.exists(backbone_file):
    print('downloading backbone')
    os.system('wget https://github.com/leondgarse/keras_efficientnet_v2/releases/download/effnetv2_pretrained/' + backbone_file + ' -O ' + backbone_file)
model = efficientnet.EfficientNetV2S(pretrained=backbone_file,dropout=1e-6, num_classes=0, include_preprocessing = True)
model.summary()

In [ ]:
batch_size = 32
image_size = (384, 384)
image_batch = []
image_paths = []
output_path = './dataset'

source_path = './extracted'
images = glob(f'{source_path}/*.jpg')

crop = False

for i, image_path in enumerate(images):
    if not os.path.exists(os.path.join(output_path, os.path.basename(image_path).replace('.jpg', '.txt'))):
        # copy from source to output
        #print(f'copying {image_path.replace(".jpg", ".txt")} to {os.path.join(output_path, os.path.basename(image_path).replace(".jpg", ".txt"))}')
        os.system(f'cp {image_path.replace(".jpg", ".txt")} {os.path.join(output_path, os.path.basename(image_path).replace(".jpg", ".txt"))}')
    if os.path.exists(os.path.join(output_path, os.path.basename(image_path).replace('.jpg', '.npy'))):
        continue
    image = cv2.imread(image_path)
    if crop:
        image_width = image.shape[1]
        image_height = image.shape[0]
        image = image[image_height // 2: image_height, image_width // 4: image_width - image_width // 4]
    image = cv2.resize(image, image_size)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.astype(np.float32)
    image_batch.append(image)
    image_paths.append(image_path)
    if len(image_batch) == batch_size or i == len(images) - 1:
        image_batch = np.array(image_batch)
        predictions = model.predict(image_batch)
        for j, image_path in enumerate(image_paths):
            #print(f'{image_path} - {predictions[j][0][0]}')
            np.save(os.path.join(output_path, os.path.basename(image_path).replace('.jpg', '.npy')), predictions[j])
        image_batch = []
        image_paths = []
        

In [ ]:
epoch = 0
lr = 1e-3